In [ ]:
import os, warnings
import time as tm
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
from functools import partial
from dynamics import Dynamics
from blr_def import BayesLR

# Data and Global Settings
a0 = 1.; b0 = 1e+2; M = 100; batchsize = 50; # batchsize = -1;
n_iter = 250; n_round = 50
# n_iter = 100; n_round = 10
datastr = 'a0 = {:.2e}; b0 = {:.2e}; M = {:d}; batchsize = {:d};'.format(a0, b0, M, batchsize)
######################
data = loadmat('./data/covertype.mat')
X_input = data['covtype'][:, 1:]
X_input = np.hstack([X_input, np.ones([X_input.shape[0], 1])])
y_input = data['covtype'][:, 0]
y_input[y_input == 2] = 0 # !!!
# split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_input, y_input, test_size=0.2, random_state=42)
fullsize, featsize = X_train.shape
shuffle_ids = np.random.permutation(fullsize)
dataName = 'covtype'
######################
dirname = './blr_res_' + dataName + '/'
if not os.path.isdir(dirname): os.makedirs(dirname)
model = BayesLR(featsize=featsize, M=M, a0=a0, b0=b0)
get_logp = partial(model.get_logp, fullsize=fullsize)
print('Feature dimension: {:d}'.format(featsize)); print('Tr/Ts Data size: {:d}/{:d}'.format(fullsize, X_test.shape[0]))
######################

In [ ]:
lwth = 2
def blr_run(pm):
    pmstr_full = '; '.join([k + ' = ' + repr(getattr(pm, k)) for k in dir(pm) if not k.startswith('__')])
    pmstr_lite = '_'.join((pm.dnType, pm.accType, pm.bwType if hasattr(pm, 'bwType') else 'void', pm.optType))
    appd = -1
    while True:
        appd += 1; subfilename = 'M{:d}_Bs{:d}_{}_{:d}.txt'.format(M, batchsize, pmstr_lite, appd)
        filename = dirname + subfilename
        if not os.path.exists(filename): break
    print('\n'.join((datastr, pmstr_full, 'Results written in file:\n"{}"\n'.format(filename))))
    
    plt_iter = range(n_iter, n_iter*n_round+1, n_iter); plt_val = np.zeros((5, n_round))
    op_samples, dninfo = Dynamics(pm.dnType, pm).evolve(model.latvar, get_logp=get_logp)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        with open(filename, 'w') as fid:
            fid.write(datastr + '\n' + pmstr_full + '\n')
            fid.write(73*'-' + '\n')
            fid.write(' iter accuracy    log-llh    |     bwmax        bwmed      bandwidth\n')
            fid.write(29*'-' + '+' + 43*'-' + '\n')
            print('Total Round {:d}:'.format(n_round)),
            for it in range(n_round):
                print(' {:d}'.format(it+1)),
                for j in range(n_iter):
                    total_iter = it*n_iter + j
                    batch_ids = shuffle_ids[[i % fullsize for i in range(total_iter*batchsize, (total_iter+1)*batchsize)]]
                    if j == n_iter-1: break;
                    sess.run(op_samples, {model.X_train: X_train[batch_ids], model.Y_train: y_train[batch_ids]})
                _, bandw, bwmed, bwmax = sess.run([op_samples, dninfo.bandw, dninfo.bwmed, dninfo.bwmax],\
                                                 {model.X_train: X_train[batch_ids], model.Y_train: y_train[batch_ids]})
                acc, llh = sess.run([model.acc, model.llh], {model.X_test: X_test, model.Y_test: y_test})
                plt_val[:,it] = [acc, llh, bwmax, bwmed, bandw]
                fid.write('{:5d} {:.6f} {:.6e} | {:.6e} {:.6e} {:.6e}\n'.format((it+1)*n_iter, *plt_val[:,it]))
            print('\n')
    return subfilename

def loadfile(collist, subfilename):
    records = []; iters = []
    with open(dirname + subfilename, 'r') as fid:
        for i in range(5): fid.readline()
        for line in fid:
            strs = line.split()
            records.append([float(strs[i]) for i in collist])
            iters.append(int(strs[0]))
    return np.array(records), np.array(iters)

def figmk(colnames, errbar, *cmdlist):
    coldict = {'acc': 1, 'llh': 2, 'diam': 4, 'med': 5, 'bw': 6}
    colinvdict = {1: 'accuracy', 2: 'log-likelihood', 4: 'sample maxdist', 5: 'sample median', 6: 'bandwidth'}
    collist = [coldict[name] for name in colnames]
    nfig = len(colnames)
    fig, ax = plt.subplots(1, nfig, sharex=True, figsize=(4*nfig+2, 4))
    if nfig == 1: ax = [ax]
    for cmd in cmdlist:
        if type(cmd[2]) == str:
            records, iters = loadfile(collist, cmd[2])
            for i in range(nfig): ax[i].plot(iters, records[:,i], cmd[1], label=cmd[0], linewidth=lwth)
        elif type(cmd[2]) == list:
            nfile = len(cmd[2])
            rec0, iters = loadfile(collist, cmd[2][0])
            reclist = np.zeros((nfile,) + rec0.shape)
            reclist[0] = rec0
            for n in range(1, nfile):
                reclist[n], iters_n = loadfile(collist, cmd[2][n])
                if np.any(iters_n != iters):
                    warnings.warn('Iteration not aligned for files with label "%s" (n = %d)! Entry ignored'%(cmd[0], n))
                    break
            else:
                means = np.sum(reclist, axis=0)/nfile
                if not errbar:
                    for i in range(nfig): ax[i].plot(iters, means[:,i], cmd[1], label=cmd[0], linewidth=lwth)
                else:
                    errs = np.sqrt(np.sum(reclist**2, axis=0)/nfile - means**2)
                    for i in range(nfig): ax[i].errorbar(iters, means[:,i], errs[:,i], fmt=cmd[1], label=cmd[0], elinewidth=lwth)
    for i in range(nfig): ax[i].set_ylabel(colinvdict[collist[i]]); ax[i].set_xlabel('iteration'); ax[i].legend()
    return fig, ax

# SVGD

In [ ]:
class PM: 
    dnType = 'SVGD'; dnNormalize = False
    accType = 'wgd';
    bwType = 'med'; optType = 'adag'; optRemem = .9; optFudge = 1e-6; stepsize = 3e-2
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('SVGD-WGD', 'r-', subfname))

In [ ]:
class PM: 
    dnType = 'SVGD'; dnNormalize = False
    accType = 'po'; accRemem = .7; accExpo = 1.; accNoise = 1e-7
    bwType = 'med'; optType = 'gd'; stepsize = 3e-6
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('SVGD-PO', 'k:', subfname))

In [ ]:
class PM:
    dnType = 'SVGD'; dnNormalize = False
    accType = 'wnag'; accRemem = 3.9
    optType = 'sgd'; optExpo = .9; stepsize = 1e-6
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('SVGD-WAG', 'b--', subfname))

In [ ]:
class PM: 
    dnType = 'SVGD'; dnNormalize = False
    accType = 'wnes1'; accHessBnd = 3e+2; accShrink = .2
    optType = 'sgd'; optExpo = .8; stepsize = 3e-4
    bwType = 'med'; # bwType = 'he'; bwSubType = 'h'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('SVGD-WNes', 'g-.', subfname))

# Blob

In [ ]:
class PM:
    dnType = 'Blob'
    accType = 'wgd'
    optType = 'gd'; stepsize = 1e-6
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('Blob-WGD', 'r-', subfname))

In [ ]:
class PM:
    dnType = 'Blob'
    accType = 'po'; accExpo = 1.; accRemem = .7; accNoise = 1e-7
    optType = 'gd'; stepsize = 3e-7
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('Blob-PO', 'k:', subfname))

In [ ]:
class PM:
    dnType = 'Blob'
    accType = 'wnag'; accRemem = 3.9
    optType = 'sgd'; optExpo = .9; stepsize = 1e-6
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('Blob-WAG', 'b--', subfname))

In [ ]:
class PM:
    dnType = 'Blob'
    accType = 'wnes1'; accHessBnd = 1e+4; accShrink = .2
    optType = 'sgd'; optExpo = .9; stepsize = 1e-5
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('Blob-WNes', 'g-.', subfname))

# GFSD

In [ ]:
class PM:
    dnType = 'GFSD'
    accType = 'wgd'
    optType = 'gd'; stepsize = 1e-6
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('GFSD-WGD', 'r-', subfname))

In [ ]:
class PM:
    dnType = 'GFSD'
    accType = 'po'; accExpo = 1.; accRemem = .7; accNoise = 1e-7
    optType = 'gd'; stepsize = 3e-7
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('GFSD-PO', 'k:', subfname))

In [ ]:
class PM:
    dnType = 'GFSD'
    accType = 'wnag'; accRemem = 3.9
    optType = 'sgd'; optExpo = .9; stepsize = 1e-6
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('GFSD-WAG', 'b--', subfname))

In [ ]:
class PM:
    dnType = 'GFSD'
    accType = 'wnes1'; accHessBnd = 1e+4; accShrink = .2
    optType = 'sgd'; optExpo = .9; stepsize = 1e-5
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('GFSD-WNes', 'g-.', subfname))

# GFSF

In [ ]:
class PM:
    dnType = 'GFSF'; dnRidge = 1e-5
    accType = 'wgd'
    optType = 'gd'; stepsize = 1e-6
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('GFSF-WGD', 'r-', subfname))

In [ ]:
class PM:
    dnType = 'GFSF'; dnRidge = 1e-5
    accType = 'po'; accExpo = 1.; accRemem = .7; accNoise = 1e-7
    optType = 'gd'; stepsize = 3e-7
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('GFSF-PO', 'k:', subfname))

In [ ]:
class PM:
    dnType = 'GFSF'; dnRidge = 1e-5
    accType = 'wnag'; accRemem = 3.9
    optType = 'sgd'; optExpo = .9; stepsize = 1e-6
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('GFSF-WAG', 'b--', subfname))

In [ ]:
class PM:
    dnType = 'GFSF'; dnRidge = 1e-5
    accType = 'wnes1'; accHessBnd = 1e+4; accShrink = .2
    optType = 'sgd'; optExpo = .9; stepsize = 1e-5
    bwType = 'med'
t0 = tm.time(); subfname = blr_run(PM()); t1 = tm.time(); print(t1-t0)
fig, ax = figmk(['acc', 'llh', 'diam', 'bw'], False, ('GFSF-WNes', 'g-.', subfname))